In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.13.1
Torchvision Version:  0.14.1


In [2]:
data_dir = "/media/abdul/8266fdc5-61f6-47d6-9e82-aa0e56f4b319/Ilyas/classification/food"

num_classes = 102
batch_size = 32
num_epochs = 10

model = models.mobilenet_v2(pretrained=True)
# Update model output size to 2.
model.classifier = nn.Linear(1280, num_classes)
model

/home/abdul/miniconda3/envs/detr/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/abdul/miniconda3/envs/detr/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [3]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
       # print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


In [5]:
train_dataset_size = len(image_datasets['train'])
val_dataset_size = len(image_datasets['val'])

print("Number of training samples: ", train_dataset_size)
print("Number of validation samples: ", val_dataset_size)


Number of training samples:  73629
Number of validation samples:  8181


In [6]:
model_ft = model.to(device)

params_to_update = []

for name, param in model_ft.named_parameters():
    param.requires_grad == False
    

for name,param in model_ft.named_parameters():

    if 'features.17' in name:
        param.requires_grad == True
        params_to_update.append(param)        

    if 'features.18' in name:
        param.requires_grad == True
        params_to_update.append(param)
        
    if 'classifier' in name:
        param.requires_grad == True
        params_to_update.append(param)
        

In [7]:
optimizer_ft = optim.SGD(params_to_update, lr=0.0005, momentum=0.9)

criterion = nn.CrossEntropyLoss()

model_ft, hist = train_model(model_ft, 
                             dataloaders_dict, 
                             criterion, 
                             optimizer_ft, 
                             num_epochs=25)

----------
train Loss: 2.9874 Acc: 0.3675
val Loss: 2.1110 Acc: 0.5105

----------
train Loss: 1.9491 Acc: 0.5300
val Loss: 1.7657 Acc: 0.5590

----------
train Loss: 1.7061 Acc: 0.5750
val Loss: 1.6298 Acc: 0.5837

----------
train Loss: 1.5772 Acc: 0.6009
val Loss: 1.5451 Acc: 0.6052

----------
train Loss: 1.4856 Acc: 0.6205
val Loss: 1.5043 Acc: 0.6134

----------
train Loss: 1.4139 Acc: 0.6364
val Loss: 1.4551 Acc: 0.6236

----------
train Loss: 1.3544 Acc: 0.6491
val Loss: 1.4254 Acc: 0.6299

----------
train Loss: 1.3024 Acc: 0.6602
val Loss: 1.4014 Acc: 0.6354

----------
train Loss: 1.2578 Acc: 0.6708
val Loss: 1.3750 Acc: 0.6393

----------
train Loss: 1.2166 Acc: 0.6808
val Loss: 1.3615 Acc: 0.6469

----------
train Loss: 1.1810 Acc: 0.6899
val Loss: 1.3556 Acc: 0.6474

----------
train Loss: 1.1467 Acc: 0.6965
val Loss: 1.3479 Acc: 0.6525

----------
train Loss: 1.1155 Acc: 0.7061
val Loss: 1.3279 Acc: 0.6543

----------
train Loss: 1.0844 Acc: 0.7138
val Loss: 1.3329 Acc: 

In [ ]:
img_test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
test_data = torchvision.datasets.ImageFolder(data_dir + '\\val',transform=img_test_transforms)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          shuffle=False, num_workers=4)

In [ ]:
class_names = image_datasets['train'].classes
print(class_names)

In [ ]:
# test-the-model
classes = ['fire lily', 'canterbury bells', 'bolero deep blue', 'pink primrose',
           'mexican aster', 'prince of wales feathers', 'moon orchid', 'globe-flower', 
           'grape hyacinth', 'corn poppy', 'toad lily', 'siam tulip', 'red ginger', 'spring crocus',
           'alpine sea holly', 'garden phlox', 'globe thistle', 'tiger lily', 'ball moss', 'love in the mist',
           'monkshood', 'blackberry lily', 'spear thistle', 'balloon flower', 'blanket flower', 'king protea', 
           'oxeye daisy', 'yellow iris', 'cautleya spicata', 'carnation', 'silverbush', 'bearded iris', 'black-eyed susan', 
           'windflower', 'japanese anemone', 'giant white arum lily', 'great masterwort', 'sweet pea', 'tree mallow', 
           'trumpet creeper', 'daffodil', 'pincushion flower', 'hard-leaved pocket orchid', 'sunflower', 'osteospermum', 
           'tree poppy', 'desert-rose', 'bromelia', 'magnolia', 'english marigold', 'bee balm', 'stemless gentian', 'mallow', 
           'gaura', 'lenten rose', 'marigold', 'orange dahlia', 'buttercup', 'pelargonium', 'ruby-lipped cattleya', 'hippeastrum', 
           'artichoke', 'gazania', 'canna lily', 'peruvian lily', 'mexican petunia', 'bird of paradise', 'sweet william', 
           'purple coneflower', 'wild pansy', 'columbine', "colt's foot", 'snapdragon', 'camellia', 'fritillary', 
           'common dandelion', 'poinsettia', 'primula', 'azalea', 'californian poppy', 'anthurium', 'morning glory', 
           'cape flower', 'bishop of llandaff', 'pink-yellow dahlia', 'clematis', 'geranium', 'thorn apple', 'barbeton daisy', 
           'bougainvillea', 'sword lily', 'hibiscus', 
           'lotus', 'cyclamen', 'foxglove', 'frangipani', 'rose', 
           'watercress', 'water lily', 'wallflower', 'passion flower', 'petunia']
model_ft.eval()  # it-disables-dropout
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
       
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        i=0
        for label in labels:
            print("input",classes[label],"output",classes[predicted[i]])
           
            i=i+1

        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))
    print('correct: {:d}  total: {:d}'.format(correct, total))